# Vector Store Creation from Documents

In [11]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import faiss

In [3]:
# Add your PDF file paths here
pdf_files_path =['C:\\Users\\rusla\\AppData\\Local\\Temp\\file_1.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_2.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_3.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_4.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_5.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_6.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_7.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_8.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_9.pdf',
 'C:\\Users\\rusla\\AppData\\Local\\Temp\\file_10.pdf']
    

In [9]:
# Function to get text from PDF documents
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        text += " ".join(page.extract_text() for page in pdf_reader.pages)
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

def embed_text(text_chunks):
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
    embeddings = embed(text_chunks)
    return embeddings.numpy()

def get_vectorstore(embeddings):
    vectorstore = faiss.IndexFlatL2(embeddings.shape[1])
    vectorstore.add(embeddings)
    return vectorstore

In [12]:
# Get PDF text and split into chunks
raw_text = get_pdf_text(pdf_files_path)

In [13]:
text_chunks = get_text_chunks(raw_text)

Created a chunk of size 1144, which is longer than the specified 1000
Created a chunk of size 1212, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000


In [14]:
# Create embeddings and vector store
embeddings = embed_text(text_chunks)

In [15]:
vectorstore = get_vectorstore(embeddings)

In [16]:
def save_vectorstore(vectorstore, filename):
    faiss.write_index(vectorstore, filename)

save_vectorstore(vectorstore, "vectorstore.faiss")

In [17]:
def load_vectorstore(filename):
    return faiss.read_index(filename)

loaded_vectorstore = load_vectorstore("vectorstore.faiss")